In [1]:
import logging
logging.basicConfig(level=logging.INFO)
import os
from dotenv import load_dotenv
from tavily import TavilyClient
load_dotenv()
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [2]:

import os
from ingestion import load_documents

base_path = os.path.abspath("..")
#DOCS = load_documents(base_path)

#print(f"Loaded {len(DOCS)} documents.") 

from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_documents(docs, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunked = []
    for doc in docs:
        chunks = splitter.split_text(doc["text"])
        for i, chunk in enumerate(chunks):
            chunked.append({
                "file": doc["file"],
                "chunk_id": i,
                "text": chunk
            })
    return chunked

DOCS_RAW = load_documents(base_path)       # one entry per file
DOCS = chunk_documents(DOCS_RAW)           # one entry per chunk


INFO:numexpr.utils:NumExpr defaulting to 10 threads.


In [3]:
from langchain.tools import tool  
from langchain_openai import ChatOpenAI  
from crewai_tools import DOCXSearchTool, WebsiteSearchTool

@tool(
    "wordcount_tool",
    description="Use this tool ONLY when the user asks for a word count. Input can be a filename or text. If a filename is provided, it must match a loaded document. Returns the exact word count."
)
def wordcount_tool(input_data: str) -> str:
    logging.info("Tool Word count used")
    for d in DOCS:
        if d["file"] == input_data:
            return f"{input_data} word count: {len(d['text'].split())}"
    for d in DOCS:
        if input_data.lower() in d["file"].lower():
            return f"{d['file']} word count: {len(d['text'].split())}"
    return f"Text word count: {len(input_data.split())}"


@tool(
    "websearch_tool",
    description="Use this tool when the user asks to search the web in general. Input is a plain natural-language query. Returns extracted results from real online sources."
)
def websearch_tool(query: str) -> str:
    logging.info("Tool Web Search used (Tavily)")
    try:
        result = tavily.search(query, max_results=5)
        if not result or "results" not in result:
            return "No results found."
        
        formatted = []
        for item in result["results"]:
            title = item.get("title", "No title")
            url = item.get("url", "No URL")
            content = item.get("content", "")[:400]  # first 400 chars
            formatted.append(f"- {title}\n  {url}\n  {content}\n")

        return "\n".join(formatted)
    
    except Exception as e:
        return f"Error during web search: {e}"


@tool(
    "news_search_tool",
    description="Use this tool only when the user asks specifically for news articles. Input is a plain natural-language query. Searches BBC, CNN, Reuters and NYTimes for relevant articles."
)
def news_search_tool(query: str) -> str:
    logging.info("Tool News Search used")
    search_query = (
        f"site:bbc.com OR site:cnn.com OR site:reuters.com "
        f"OR site:nytimes.com {query}"
    )
    try:
        return WebsiteSearchTool().run(search_query) or "No news results."
    except Exception as e:
        return f"Error during news search: {e}"




In [4]:
# Shared cache for DOCXSearchTool instances
DOCX_TOOL_CACHE = {}

def get_docx_tool(file_path):
    if file_path not in DOCX_TOOL_CACHE:
        DOCX_TOOL_CACHE[file_path] = DOCXSearchTool(docx=file_path)
    return DOCX_TOOL_CACHE[file_path]

def resolve_doc_path(name_or_path: str):
    needle = name_or_path.strip().strip("'").strip('"').lower()
    for d in DOCS_RAW:
        path = d["file"]
        if needle == os.path.basename(path).lower() or needle in path.lower():
            return path
    return None

def parse_file_and_query(raw: str):
    if "|" not in raw:
        return None, raw.strip().strip("'").strip('"')
    file_part, q = [x.strip() for x in raw.split("|", 1)]
    return resolve_doc_path(file_part), q.strip().strip("'").strip('"')

@tool("keyword_search_tool", description="Search keyword inside ONE DOCX. Input: filename.docx | keyword")
def keyword_search_tool(raw: str) -> str:
    logging.info("Tool KW SINGLE DOC used")
    doc_path, keyword = parse_file_and_query(raw)
    if not doc_path:
        return "Provide: filename.docx | keyword (file not found)"
    try:
        tool = get_docx_tool(doc_path)
        hits = tool.run(keyword.lower())
        return f"\n📄 {os.path.basename(doc_path)}\n{hits}" if hits else "No matches in that document."
    except Exception as e:
        logging.warning(f"Keyword search error in {doc_path}: {e}")
        return f"Error during keyword search: {e}"

@tool("vector_search_tool", description="Semantic search inside ONE DOCX. Input: filename.docx | query")
def vector_search_tool(raw: str) -> str:
    logging.info("Tool VECTOR SINGLE DOC used")
    doc_path, query = parse_file_and_query(raw)
    if not doc_path:
        return "Provide: filename.docx | semantic query (file not found)"
    try:
        tool = get_docx_tool(doc_path)
        hits = tool.run(query)
        return f"\n📄 {os.path.basename(doc_path)}\n{hits}" if hits else "No semantic matches in that document."
    except Exception as e:
        logging.warning(f"Vector search error in {doc_path}: {e}")
        return f"Error during vector search: {e}"


In [17]:
from crewai_tools import FileWriterTool
@tool(
    "file_write_tool",
    description="Use this tool to write text to a file. Input must be: filename | content. The file will be created in the working directory."
)
def file_write_tool(input_str: str) -> str:
    """Writes content into a file using CrewAI's FileWriterTool."""
    logging.info("Tool File Writer used")

    # Check format
    if "|" not in input_str:
        return (
            "Format error. Use: filename | content\n"
            "Example: notes.txt | This is the content to write."
        )

    filename, content = [
        x.strip().strip("'").strip('"')
        for x in input_str.split("|", 1)
    ]

    try:
        writer = FileWriterTool()
        path = writer._run(filename=filename, text=content)
        return f"File written successfully: {path}"
    except Exception as e:
        return f"Error writing file: {e}"


@tool(
    "summarize_tool",
    description="Use this tool ONLY when the user asks for a summary of a specific document. Input must be the exact filename or any part of it. The tool loads the document text internally and returns a concise 10-sentence summary."
)
def summarize_tool(input_data: str) -> str:
    logging.info("Tool Summariser used")
    clean = input_data.strip().strip("'").strip('"').lower()

    # Search in DOCS_RAW (per-file structure)
    for d in DOCS_RAW:
        fname = d["file"].lower()
        if clean == os.path.basename(fname) or clean in fname:
            input_data = d["text"]
            break
    else:
        return f"Document '{clean}' not found in loaded documents."

    try:
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        summary = llm.invoke(f"Write a summary in 10 sentences about :\n\n{input_data}").content
        return summary or "LLM returned an empty summary. Try again later."
    except Exception as e:
        logging.error(f"LLM invocation failed: {e}")
        return f"Error during summary generation: {e}"




@tool(
    "compare_texts_tool",
    description="Use this tool to compare two texts. Input must be: textA | textB. The tool returns a detailed comparison of similarities and differences."
)
def compare_texts_tool(input_str: str) -> str:
    logging.info("Tool Compare Texts used")

    # Validate the format
    if "|" not in input_str:
        return (
            "Format error. Use: textA | textB\n"
            "Example: 'policy text A' | 'policy text B'"
        )

    textA, textB = [x.strip().strip("'").strip('"') for x in input_str.split("|", 1)]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    prompt = f"""
Compare the two texts below in a clear, structured way.

Your comparison MUST include:
1. A summary of what each text is about.
2. A list of key similarities.
3. A list of key differences.
4. Any differences in tone, structure, detail, or perspective.
5. A short final conclusion comparing their overall purpose and message.

TEXT A:
{textA}

TEXT B:
{textB}
"""

    return llm.invoke(prompt).content

@tool(
    "format_text_tool",
    description="Use this tool to clean, rewrite, and professionally format any text using an LLM. Input is raw text; output is a polished, well-structured version."
)
def format_text_tool(input_data: str) -> str:
    """Formats and rewrites text using an LLM to improve clarity, flow, and structure."""
    logging.info("Tool Format Text used")

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    prompt = f"""
Rewrite and clean the following text. Your output MUST:

- Fix spacing, punctuation, and structure.
- Keep meaning identical.
- Improve clarity and flow.
- Use consistent tone and formatting.
- Break into clean sentences and paragraphs where needed.

TEXT TO FORMAT:
{input_data}
"""

    return llm.invoke(prompt).content



In [19]:
from langchain.agents import create_agent

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.1,
    max_tokens=1000,
    timeout=30
)

reader_agent = create_agent(
    model,
    tools=[wordcount_tool, keyword_search_tool, vector_search_tool, news_search_tool, websearch_tool],
    system_prompt=(
        "You are SEARCHER, a retrieval specialist. "
        "Your ONLY job is to decide which retrieval tool is appropriate and then call it. "
        "Follow these strict rules:\n\n"
        
        "1. Use wordcount_tool ONLY when the user explicitly asks for a word count.\n"
        "2. Use keyword_search_tool ONLY for literal keyword searches INSIDE a specific DOCX file. "
        "Input MUST be formatted as: filename.docx | keyword.\n"
        "3. Use vector_search_tool ONLY for semantic search INSIDE a specific DOCX file. "
        "Input MUST be formatted as: filename.docx | semantic query.\n"
        "4. Use news_search_tool ONLY when the user asks about recent news or news articles.\n"
        "5. Use websearch_tool for general internet questions that are not news-specific.\n\n"
        
        "Do NOT summarize. Do NOT rewrite. Do NOT generate your own content. "
        "Your job is retrieval ONLY. If the user request is not about retrieving information, you MUST NOT answer directly—just choose the correct tool."
    )
)

writer_agent = create_agent(
    model,
    tools=[summarize_tool, compare_texts_tool, format_text_tool, file_write_tool],
    system_prompt=(
        "You are WriterBot, a writing, formatting, and document-editing specialist. "
        "Your job is to select exactly ONE tool for each request.\n\n"
        
        "TOOL SELECTION RULES:\n\n"

        "1. summarize_tool\n"
        "- Use this tool when the user wants a summary of a document or a block of text.\n"
        "- Input is plain text or a filename.\n\n"

        "2. compare_texts_tool\n"
        "- Use this tool ONLY when the user provides two texts OR two filenames separated by '|'.\n"
        "- Example format: textA | textB\n"
        "- The goal is to compare similarities and differences.\n\n"

        "3. format_text_tool\n"
        "- Use this tool when the user wants text cleaned, rewritten, improved, or formatted.\n"
        "- Input is a single block of text.\n\n"

        "4. file_write_tool\n"
        "- Use this tool when the user wants to write text into a file.\n"
        "- Input MUST be: filename | content\n"
        "- Example: notes.txt | This is the text to save.\n\n"

        "ROUTING RULES:\n\n"
        "- Choose only ONE tool for each request.\n"
        "- Never attempt to answer using the model alone.\n"
        "- Never combine or chain tools.\n"
        "- If the user request does not follow the correct input format for any tool, ask the user to rewrite the request.\n"
        "- Your responses must always be a direct tool call.\n\n"

        "You do not generate text directly. You only select tools."
    )
)

In [26]:
from langchain.tools import tool

def extract_output(result):
    try:
        # LangChain agent output format: result["messages"] is a list of messages
        if isinstance(result, dict) and "messages" in result:
            messages = result["messages"]

            # 1. Look for last ToolMessage (tool output)
            for msg in reversed(messages):
                if isinstance(msg, dict):
                    if msg.get("type") == "tool" and msg.get("content"):
                        return msg["content"]
                elif hasattr(msg, "type") and msg.type == "tool" and hasattr(msg, "content"):
                    return msg.content

            # 2. Fall back to last assistant (AI) message
            for msg in reversed(messages):
                if isinstance(msg, dict):
                    if msg.get("role") == "assistant" and msg.get("content"):
                        return msg["content"]
                elif hasattr(msg, "role") and msg.role == "assistant" and hasattr(msg, "content"):
                    return msg.content

            return "No output message found."

        # Fallback: single message or AIMessage object
        if hasattr(result, "content"):
            return result.content

        return str(result)

    except Exception as e:
        return f"Failed to extract output: {e}"




@tool(
    "call_reader",
    description="Call the Reader Agent to retrieve information."
)
def call_reader(query: str):
    result = reader_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return extract_output(result)


@tool(
    "call_writer",
    description="Call the Writer Agent to summarize or rewrite content."
)
def call_writer(query: str):
    result = writer_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return extract_output(result)


In [27]:
root_agent = create_agent(
    model,
    tools=[call_reader, call_writer],
    system_prompt=(
        "You are the SUPERVISOR. Your only job is to choose whether to use the reader or writer agent based on the user's request.\n\n"
        
        "ROUTING RULES:\n\n"
        "- If the request involves summarizing, shortening, condensing, explaining, rewriting, formatting, cleaning, improving text, or comparing two texts → ALWAYS call `call_writer`.\n"
        "- If the request involves retrieving, searching, looking up, finding, keyword matching, semantic understanding, reading documents, web queries, or news → ALWAYS call `call_reader`.\n\n"

        "NEVER try to answer the user's request yourself.\n"
        "NEVER call both tools. Choose only one.\n"
        "Be strict in following the routing rules. DO NOT guess."
    )
)


In [9]:
query = "Count the number of words in 'Module 1 Lesson 3 Compliance legal and ethical considerations.docx'"
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool Word count used
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The document 'Module 1 Lesson 3 Compliance legal and ethical considerations.docx' contains 113 words.


In [28]:
query = "Summarize the document 'Module 1 Lesson 3 Compliance legal and ethical considerations.docx'"
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool Summariser used
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The course "Become a Market Leader through Reproductive and Fertility Health" includes a module focused on compliance, legal, and ethical considerations in reproductive health at work. In Lesson 3, participants explore the legal landscape in the UK and Europe, highlighting key regulations such as the Equality Act 2010, which protects against pregnancy and maternity discrimination. Landmark cases, such as those involving Mitie Ltd and Event Medical Group, illustrate the consequences of failing to comply with these laws, resulting in significant financial penalties for employers. The Employment Rights Act 1996 and the Health and Safety at Work Act 1974 further emphasize the need for workplace adjustments for reproductive health conditions. The module also covers the EU Work-Life Balance Directive and the Pregnant Workers Directive, which set minimum rights for parental leave and protections for pregnant employees. Recent legal cases demonstrate the financial and reputational damage compa

In [11]:
query = "Case Study How Company X Implemented a Reproductive and Fertility Health Guide.docx | IVF"
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool KW SINGLE DOC used
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Using Tool: Search a DOCX's content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The document "Case Study How Company X Implemented a Reproductive and Fertility Health Guide.docx" includes a discussion on IVF, focusing on cultural and religious considerations globally. It also features a case study of the NatWest Group in the UK, which provides fertility benefits, including IVF support.


In [12]:
query = "Perform a semantic search on Case Study How Company X Implemented a Reproductive and Fertility Health Guide.docx | IVF"
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool VECTOR SINGLE DOC used


Using Tool: Search a DOCX's content


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The semantic search on the document "Case Study How Company X Implemented a Reproductive and Fertility Health Guide.docx" for "IVF" reveals that the document discusses the implementation of reproductive health-inclusive workplace policies, including IVF support. It covers cultural and religious considerations, workplace impacts, and includes case studies like the NatWest Group in the UK, which provides fertility benefits such as IVF support.


In [13]:
query = "Search the web for medical ethics guidelines about IVF treatments."
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool Web Search used (Tavily)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here are some resources on medical ethics guidelines for IVF treatments:

1. **Revisiting Selected Ethical Aspects of Current Clinical In Vitro**: This article discusses the ethical responsibilities of physicians and IVF centers, focusing on principles like beneficence, non-maleficence, respect for patient autonomy, and justice. [Read more](https://pmc.ncbi.nlm.nih.gov/articles/PMC8995227/)

2. **Use of In Vitro Fertilization—Ethical Issues**: This review covers ethical aspects such as upper age limits, ownership of gametes and embryos, IVF in single women and same-sex couples, preimplantation genetic testing, social egg freezing, commercialization, public funding, and prioritization of IVF. [Read more](https://pmc.ncbi.nlm.nih.gov/articles/PMC7721055/)

3. **Global Ethics in IVF: Harmonizing Regulation, Ensuring Access**: This article proposes a WHO-convened global ethics framework to standardize IVF ethical practices by 2027, addressing disparities in access and regulation. [Read mor

In [14]:
query = "Search major any news about IVF."
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool News Search used


Using Tool: Search in a specific website


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here are some recent news highlights about IVF:

1. **Legal Cases**: There have been legal cases where companies faced repercussions for not accommodating employees undergoing IVF treatments. For instance, Mrs. Benton was dismissed after undergoing IVF treatment and suffering a miscarriage. The tribunal found she was harassed and victimized due to her IVF treatment.

2. **Corporate Policies**: Companies like NatWest Group in the UK are implementing fertility benefits, including IVF support, egg freezing, and adoption assistance. They provide paid leave for fertility treatments and have dedicated networks for fertility and baby loss support.

3. **Cultural and Religious Considerations**: Cultural and religious considerations significantly influence reproductive healthcare policies globally. Different regions have varying attitudes towards IVF and fertility treatments, affecting workplace policies and benefits.

These developments underscore the importance of inclusive reproductive healt

In [29]:
query = "Compare the following two texts | Text A: “IVF success often depends on age and medical history.” | Text B: “Fertility outcomes vary widely across age groups, with younger patients having higher chances.”."
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool Compare Texts used
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### Summary of Each Text

**Text A:**  
Text A discusses the factors influencing the success of in vitro fertilization (IVF), specifically highlighting the importance of age and medical history. It suggests that these two elements play a crucial role in determining the likelihood of a successful pregnancy through IVF.

**Text B:**  
Text B focuses on the variability of fertility outcomes based on age, emphasizing that younger patients tend to have significantly higher chances of successful IVF compared to older patients. It presents a more specific observation about the relationship between age and fertility success rates.

### Key Similarities

1. **Subject Matter:** Both texts address the topic of IVF and its success rates.
2. **Focus on Age:** Each text acknowledges the impact of age on fertility outcomes.
3. **Implication of Variability:** Both texts imply that success rates are not uniform and can differ based on certain factors.

### Key Differences

1. **Scope of Discussion:**  

In [16]:
query = "Format this text: “ivf is a complicated procesS , often involving multiple stages. sometimes employees need support but companies dont always understand how to help.”"
response = root_agent.invoke({"messages":[{"role":"user","content":query}]})
print(extract_output(response))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Tool Format Text used
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Here is the formatted text:

"IVF is a complicated process that often involves multiple stages. Employees may require support during this time, but companies do not always understand how to provide assistance."
